In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

In [3]:
# create a map of mission date and mission number
date_mission_df = pd.read_csv('hera_ind_perf.csv').loc[:, ['mission', 'date']].drop_duplicates()
date_mission_dict = dict(date_mission_df.values)

# merge date to hera normalized individual performance, IoI, ToI, and MoI
hera_perf = pd.read_csv('Team Performance All.csv').iloc[:, 1:]
hera_perf['date'] = hera_perf['Mission'].map(date_mission_dict)

# only use hera performance for this study
hera_perf = hera_perf.loc[-hera_perf['date'].isna()].rename(
columns = {'subject':'team', 'Mission':'mission'}).loc[
    :, ['date', 'mission', 'team', 'normalized_perf', 'ToT', 'MoT']]
hera_perf['datetime'] = hera_perf['date'].apply(lambda x: dt.strptime(x, '%m/%d/%y'))

In [5]:
# SMM1
hera_smm1 = pd.read_csv('hera_smm1_survey.csv').drop(columns = ['Campaign', 'Mission', 'Session']).rename(
columns = {'StartDate':'date', 'Q2.1':'role'})
hera_smm1['team'] = hera_smm1['role'].map({1:'Rob', 2:'Rob', 3:'Rob', 4:'Eng', 5:'Eng', 6:'Eng',
                                        7:'Geo', 8:'Geo', 9:'Geo', 10:'HF', 11:'HF', 12:'HF'})
hera_smm1['datetime'] = hera_smm1['date'].apply(lambda x: dt.strptime(x, '%m/%d/%y'))

# SMM2
hera_smm2 = pd.read_csv('hera_smm2_survey.csv').drop(columns = ['Campaign', 'Mission', 'Session']).rename(
columns = {'StartDate':'date', 'Q2.1':'role'})
hera_smm2['team'] = hera_smm2['role'].map({1:'Rob', 2:'Rob', 3:'Rob', 4:'Eng', 5:'Eng', 6:'Eng',
                                        7:'Geo', 8:'Geo', 9:'Geo', 10:'HF', 11:'HF', 12:'HF'})
hera_smm2['datetime'] = hera_smm2['date'].apply(lambda x: dt.strptime(x, '%m/%d/%y'))

# exclude dates where either performance or SMM surveys is missing
hera_perf_date = list(hera_perf.datetime.unique())
hera_smm1_date = list(hera_smm1.datetime.unique())
hera_smm2_date = list(hera_smm2.datetime.unique())

mutual_date = list(set(hera_perf_date).intersection(set(hera_smm1_date)).intersection(set(hera_smm2_date)))

hera_perf = hera_perf.loc[hera_perf.datetime.isin(mutual_date), :]

hera_norm_perf = hera_perf.loc[:, ['datetime', 'team', 'normalized_perf']].dropna()
hera_ToT_perf = hera_perf.loc[:, ['datetime', 'team', 'ToT']].dropna()
hera_MoT_perf = hera_perf.loc[:, ['datetime', 'team', 'MoT']].dropna()

hera_smm1 = hera_smm1.loc[hera_smm1.datetime.isin(mutual_date), :].drop(columns = ['date', 'role'])
hera_smm2 = hera_smm2.loc[hera_smm2.datetime.isin(mutual_date), :].drop(columns = ['date', 'role'])

In [6]:
# compute SMM block diagonal matrix csv for ALAAM
def bd_matrix_team(smm_df, perf_df, perf_name, smm_name):
    try:
        smm_df = smm_df.groupby(['datetime', 'team']).mean().reset_index().sort_values(
            by = ['datetime', 'team'])
        mission = list(smm_df['datetime'].unique())
        
        hera_smm = []
        for i in mission:
            m = smm_df[smm_df['datetime'] == i].iloc[:, 2:].to_numpy()
            row, col = m.shape
            smm = []
            for j in range(row - 1):
                for k in list(range(j + 1, row)):
                    temp = m[j] - m[k]
                    smm.append([j + 1, k + 1, np.sqrt(np.dot(temp.T, temp)) / 10 * np.sqrt(10)])
            hera_smm.append((i, smm))
        
        hera_link = {}
        for i in range(len(hera_smm)):
            m = hera_smm[i][1]
            smm = [x[2] for x in m]
            avg_smm = sum(smm) / len(smm)
            link = [x[:2] for x in m if x[2] < avg_smm]
            hera_link[[x[0] for x in hera_smm][i]] = link 
            
        adjacency_list = []
        for i in mission:
            edge_list = hera_link[i]
            size = smm_df[smm_df['datetime'] == i].shape[0]
            adjacency = [[0]*size for _ in range(size)]
            for sink, source in edge_list:
                adjacency[sink - 1][source - 1] = 1
            for i in range(size):
                for j in range(size):
                    t = adjacency[i][j]
                    adjacency[j][i] = t
            adjacency_list.append(adjacency)
            
        mat_list = []
        for i in range(len(adjacency_list)):
            mat = adjacency_list[i]
            front_sum = 0
            back_sum = 0
            for j in range(i):
                prev_mat = adjacency_list[j]
                front_sum = front_sum + np.array(prev_mat).shape[0]
            for k in range(i + 1, len(adjacency_list)):
                post_mat = adjacency_list[k]
                back_sum = back_sum + np.array(post_mat).shape[0]
            mat_list.append(np.block([np.zeros((np.array(mat).shape[0], front_sum)), 
                              np.array(mat), np.zeros((np.array(mat).shape[0], back_sum))]))
            
        concat_mat = mat_list[0]
        for i in list(range(1, len(mission))):
            concat_mat = np.concatenate([concat_mat, mat_list[i]])
        
        pd.DataFrame(concat_mat).astype('int').to_csv(f'{smm_name}_{perf_name}_bd_matrix_team.csv')
        
    except Exception as e:
        print(e)

In [7]:
perf_list = [[hera_norm_perf, 'normalized_perf'], [hera_ToT_perf, 'ToT'], [hera_MoT_perf, 'MoT']]
smm_list = [[hera_smm1, 'smm1'], [hera_smm2, 'smm2']]

for i in perf_list:
    for j in smm_list:
        bd_matrix_team(smm_df = j[0], perf_df = i[0], smm_name = j[1], perf_name = i[1])

In [8]:
# compute performance csv for ALAAM
    # since the response variable in ALAAM is binary, I define high performer (1) and low performer (0)
    # based on the average normalized scores of the multi-team systems
    
def perf_csv(perf_df, perf_name):
    try:
        perf_df = perf_df.groupby('datetime').mean().reset_index().rename(
        columns = {perf_name:'avg_perf'}).merge(perf_df, on = 'datetime')
        
        perf_df['binary'] = 1
        perf_df.loc[perf_df[perf_name] < perf_df['avg_perf'], 'binary'] = 0
        perf_df.drop(columns = perf_name).rename(columns = {'binary':perf_name}).loc[:, [perf_name]].to_csv(
            f'{perf_name}_team.csv')
    except Exception as e:
        print(e)

In [10]:
for i in perf_list:
    perf_csv(perf_df = i[0], perf_name = i[1])